In [11]:
# Check if mace_run_train is available
!where mace_run_train

# Or try this on Windows:
#!mace_run_train --help

C:\Users\shann\AppData\Local\Programs\Python\Python311\Scripts\mace_run_train.exe


In [ ]:
#for some reason I cannot update the database in github, so if the database is not in the folder, execute this cell
!git clone https://github.com/davkovacs/BOTNet-datasets.git

ok, so this part does not work, so instead I tried the other option from another tutorial where they write a config file.

In [26]:
'''!mace_run_train \
  --name="MACE_model" \
  --train_file="BOTNet-datasets/dataset_3BPA/train_300K.xyz" \
  --valid_fraction=0.05 \
  --forces_key="forces" \
  --energy_key="energy" \
  --test_file="BOTNet-datasets/dataset_3BPA/test_300K.xyz" \
  --E0s='{1:-13.663181292231226, 6:-1029.2809654211628, 7:-1484.1187695035828, 8:-2042.0330099956639}' \
  --model="ScaleShiftMACE" \
  --hidden_irreps='32x0e' \
  --r_max=4.0 \
  --batch_size=20 \
  --max_num_epochs=5 \
  --ema \
  --ema_decay=0.99 \
  --amsgrad \
  --default_dtype="float32" \
  --device=cpu \
  --seed=123 \
  --swa'''

'!mace_run_train   --name="MACE_model"   --train_file="BOTNet-datasets/dataset_3BPA/train_300K.xyz"   --valid_fraction=0.05   --forces_key="forces"   --energy_key="energy"   --test_file="BOTNet-datasets/dataset_3BPA/test_300K.xyz"   --E0s=\'{1:-13.663181292231226, 6:-1029.2809654211628, 7:-1484.1187695035828, 8:-2042.0330099956639}\'   --model="ScaleShiftMACE"   --hidden_irreps=\'32x0e\'   --r_max=4.0   --batch_size=20   --max_num_epochs=5   --ema   --ema_decay=0.99   --amsgrad   --default_dtype="float32"   --device=cpu   --seed=123   --swa'

So here is where the config file is created

The first line is a command that creates a new file, writing everything in the cell to a file named config_mace_3bpa.yml
- 'model' specifies the mace model varian that is going to be used -> the model used here appllies scalling and shifting to energy outputs helping with numerical stability and accuracy -> could also use MACE or BOTNet
- 'name' names the trainin run -> naming the output files, logs and checkpoints
- 'train_file' and 'test_file' are for indicating the files with the information to train and to be used to evaluate the model
- 'valid fraction' indicates how to split training data for validation, uses % of training data as validation set to monitor training progress and prevent overfitting.
- 'energy_key' and 'forces_key' tell MACE which columns in the xyz file contain energy and forces -> looks for energy property on the file and forces property (the xyz file must have this properties defined)
- 'E0s:' reference energies for isolated atoms
- 'hidden_irreps' defines de neural network architecture -> for this case 32 are the channels/features, x0e only scalar features -> uses 32 scalar features in the hidden layers
- 'r_max' cutoff radius for atomic interactions -> atoms farther than 4 angstroms apart dont interact
- 'batch size' number of structures processed together
- 'max num epochs' how long to train -> epoch is one full pass through all training data
- 'ema' and 'ema decay' exponential moving average for model weights -> keeps a smoothed versioin of the model parameters 
- 'amsgrad' improved version of Adam optimizer -> prevents overly large learning rates, more stable training
- 'swa' stochastic weight averaging -> averages multiple model checkpoints at the end of training for better performance
- 'default dtype' numerical precision
- 'device' where to run the computations -> CUDA for NVIDIA GPUs
- 'seed' random number generator seed -> random initialization every time

In [18]:
%%writefile config_mace_3bpa.yml

model: "ScaleShiftMACE"
name: "MACE_model"
train_file: "BOTNet-datasets/dataset_3BPA/train_300K.xyz"
valid_fraction: 0.05
test_file: "BOTNet-datasets/dataset_3BPA/test_300K.xyz"
energy_key: "energy"
forces_key: "forces"
E0s:
  1: -13.663181292231226
  6: -1029.2809654211628
  7: -1484.1187695035828
  8: -2042.0330099956639
hidden_irreps: "32x0e"
r_max: 4.0
batch_size: 20
max_num_epochs: 5
ema: true
ema_decay: 0.99
amsgrad: true
default_dtype: "float32"
device: cpu
seed: 123
swa: true

Overwriting config_mace_3bpa.yml


this is one way to run the training, like a command line

In [19]:
!mace_run_train --config config_mace_3bpa.yml

cuequivariance or cuequivariance_torch is not available. Cuequivariance acceleration will be disabled.
2025-10-03 22:28:53.959 INFO: ===========VERIFYING SETTINGS===========
2025-10-03 22:28:53.959 INFO: MACE version: 0.3.14
2025-10-03 22:28:53.959 INFO: Using CPU
2025-10-03 22:28:54.233 INFO: ===========LOADING INPUT DATA===========
2025-10-03 22:28:54.233 INFO: Using heads: ['Default']
2025-10-03 22:28:54.234 INFO: Using the key specifications to parse data:
2025-10-03 22:28:54.234 INFO: Default: KeySpecification(info_keys={'energy': 'energy', 'stress': 'REF_stress', 'virials': 'REF_virials', 'dipole': 'dipole', 'head': 'head', 'elec_temp': 'elec_temp', 'total_charge': 'total_charge', 'polarizability': 'polarizability', 'total_spin': 'total_spin'}, arrays_keys={'forces': 'forces', 'charges': 'REF_charges'})
2025-10-03 22:28:54.234 INFO: =============    Processing head Default     ===========
2025-10-03 22:28:54.697 WARNING: Since ASE version 3.23.0b1, using energy_key 'energy' is no

C:\Users\shann\AppData\Local\Programs\Python\Python311\Lib\site-packages\e3nn\o3\_wigner.py:10: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.dirname(__file__), 'constants.pt'))
C:\Users\shann\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\shann\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\jit\_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
C:\Users\shann\AppData\

This is another way to run the training, with this I can see what is going on with the training
- import warnings -> gives access to python warning system
- warnings.filterwarnings -> sillences all warning messages -> this is because MACE/torch can generaete many technical warninigs that clutter the output but do not affect the functionality
- from mace.cli.run_train -> the module containing the training code
- main -> main function that runs training
- as mace_run_train_main -> renames it for easer use
- import sys -> system-specific parameters and functions
- logging -> python's logging system for tracking program execution
- def -> define the training function -> this is a reusable function to train MACE models
- logging.getLogger -> celans up logging configuration -> prevents duplicate log messages and ensures a clean output
- sys.argv -> simulates a command line

In [27]:
import warnings
warnings.filterwarnings("ignore")
from mace.cli.run_train import main as mace_run_train_main
import sys
import logging

def train_mace(config_file_path):
    logging.getLogger().handlers.clear()
    sys.argv = ["program", "--config", config_file_path]  # Simulates command line
    mace_run_train_main()  # Calls the main function directly

train_mace("config_mace_3bpa.yml")

2025-10-03 23:16:52.931 INFO: ===========VERIFYING SETTINGS===========
2025-10-03 23:16:52.933 INFO: MACE version: 0.3.14
2025-10-03 23:16:52.934 INFO: Using CPU
2025-10-03 23:16:53.220 INFO: ===========LOADING INPUT DATA===========
2025-10-03 23:16:53.224 INFO: Using heads: ['Default']
2025-10-03 23:16:53.228 INFO: Using the key specifications to parse data:
2025-10-03 23:16:53.230 INFO: Default: KeySpecification(info_keys={'energy': 'energy', 'stress': 'REF_stress', 'virials': 'REF_virials', 'dipole': 'dipole', 'head': 'head', 'elec_temp': 'elec_temp', 'total_charge': 'total_charge', 'polarizability': 'polarizability', 'total_spin': 'total_spin'}, arrays_keys={'forces': 'forces', 'charges': 'REF_charges'})
2025-10-03 23:16:53.233 INFO: =============    Processing head Default     ===========
2025-10-03 23:16:53.550 WARNING: Since ASE version 3.23.0b1, using energy_key 'energy' is no longer safe when communicating between MACE and ASE. We recommend using a different key, rewriting 'en

With this the model can be used to run ASE MD. 

In [28]:
from ase import units #physical constants
from ase.md.langevin import Langevin #langevin dinamics integrator
from ase.io import read, write # read/write chemical structure files
import numpy as np 
import time #timing performance

from mace.calculators import MACECalculator # import the machine learning potential -> gives acces to mace as a "calculator" that can predict energies and forces

calculator = MACECalculator(model_paths='checkpoints/MACE_model_run-123.model', device='cpu') #loads the trained MACe model
init_conf = read('BOTNet-datasets/dataset_3BPA/test_300K.xyz', '0') # loads the starting structure for MD, loads a singe molecular configuration to start the simulation
init_conf.set_calculator(calculator) # connect the ML calculator to the atoms (en teoria se puede usar init_conf.cal = calculator pero no lo he probado)

dyn = Langevin(init_conf, 0.5*units.fs, temperature_K=310, friction=5e-3) #uses langevin dynamics (constant temperature)
#defines a function to save simulation snapshots
def write_frame():
        dyn.atoms.write('md_3bpa.xyz', append=True)
dyn.attach(write_frame, interval=50) #indicates the simulator when to save frames
dyn.run(100) #execute the molecular dynamics, each step calculates forces using mace, updates atom positions and velocities, applies thermostat and saves trajectory frame every 50 steps
print("MD finished!")

2025-10-03 23:33:42.931 INFO: Using CPU
Using head Default out of ['Default']
No dtype selected, switching to float32 to match model dtype.
MD finished!


In [29]:
import os
if os.path.exists('md_3bpa.xyz'):
    print("✓ Trajectory file created: md_3bpa.xyz")
    
    # Count how many frames were saved
    from ase.io import read
    frames = read('md_3bpa.xyz', index=':')
    print(f"✓ Saved {len(frames)} frames to md_3bpa.xyz")
else:
    print("✗ Trajectory file not created")

✓ Trajectory file created: md_3bpa.xyz
✓ Saved 6 frames to md_3bpa.xyz


In [24]:
from ase.io import read

trajectory = read('md_3bpa.xyz', index=':')
print(f"Trajectory contains {len(trajectory)} frames")
print(f"Each frame has {len(trajectory[0])} atoms")

# Print energies from the trajectory
print("\nEnergies from MD simulation:")
for i, atoms in enumerate(trajectory):
    if hasattr(atoms, 'get_potential_energy'):
        energy = atoms.get_potential_energy()
        print(f"Frame {i}: Energy = {energy:.4f} eV")

Trajectory contains 3 frames
Each frame has 27 atoms

Energies from MD simulation:
Frame 0: Energy = -17664.2285 eV
Frame 1: Energy = -17664.8047 eV
Frame 2: Energy = -17664.9277 eV
